In [9]:
import pickle
import sncosmo
import numpy as np
import scipy

In [10]:
source = pickle.load(open('../model/PLAsTiCC/SNIax/sn2005hk.source.pickle', "rb" ), encoding="latin-1" )['source']


/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_1702/4032707771.py:1: DeprecationWarning: Please use `RectBivariateSpline` from the `scipy.interpolate` namespace, the `scipy.interpolate.fitpack2` namespace is deprecated.
  source = pickle.load(open('../model/PLAsTiCC/SNIax/sn2005hk.source.pickle', "rb" ), encoding="latin-1" )['source']


In [16]:
def define_external_sncosmo_model(source):
	origmodel = sncosmo.Model(source=source)
	
	phase = np.linspace(-30.,200.,231)
	nphase = len(phase)
	wave = np.linspace(1000.,25000.,2401)
	nwave = len(wave)

	origflux = origmodel.flux(phase,wave)
	flux = np.copy(origflux)

	# smooth z and y bands which are too wiggly
	zy = np.where( (wave >= 8000.) & (wave <= 11000.))
	flux[10:,zy] = scipy.ndimage.gaussian_filter(origflux,[4.0,0.0])[10:,zy]

	# extend to late times and smooth it
	#   enforce 0.015 mag/day (= 72.4 d e-folding) late-time decline rate
	late = np.where(phase >= 50.)
	flux[late,:] *= np.exp(-(phase[late,None]-50.)/72.4)
	flux[70:,:] = scipy.ndimage.gaussian_filter(flux,[4.0,0.0])[70:,:]    

	sedmodel = sncosmo.Model(source=sncosmo.TimeSeriesSource(phase,wave,flux))
	return sedmodel

In [17]:
sedmodel = define_external_sncosmo_model(source)